In [ ]:
import pandas as pd
import numpy as np
from buckaroo.dataflow_traditional import StylingAnalysis
from buckaroo.pluggable_analysis_framework.pluggable_analysis_framework import ColAnalysis
import polars as pl
from buckaroo.polars_buckaroo import PolarsBuckarooWidget
from buckaroo.customizations.polars_analysis import PlTyping
N=500

In [ ]:
# This works with polars or pandas, these variables let us quickly swap between the two
DataFrame = pl.DataFrame
BUCKAROOWidget = PolarsBuckarooWidget

This notebook generally follows the order of [DFWhole.ts](https://github.com/paddymul/buckaroo/blob/feat/dfviewer-config/js/components/DFViewerParts/DFWhole.ts)

Starting with the simple Displayers
```js
export interface ObjDisplayerA {
  displayer: 'obj';}

export interface BooleanDisplayerA {
  displayer: 'boolean';}

export interface StringDisplayerA {
  displayer: 'string';
  max_length?: number;} 
  
export interface FloatDisplayerA {
  displayer: 'float';
  min_fraction_digits: number;
  max_fraction_digits: number;}

export interface DatetimeDefaultDisplayerA {
  displayer: 'datetimeDefault';}
export interface IntegerDisplayerA {
  displayer: 'integer';
  min_digits: number;
  max_digits: number;}

export interface DatetimeLocaleDisplayerA {
  displayer: 'datetimeLocaleString';
  locale: 'en-US' | 'en-GB' | 'en-CA' | 'fr-FR' | 'es-ES' | 'de-DE' | 'ja-JP';
  // https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Intl/DateTimeFormat/DateTimeFormat
  args: Intl.DateTimeFormatOptions;}
```

In [ ]:
#simpler Buckaroo that really shows the styling
def obj_(pkey):
    return {'primary_key_val': pkey, 'displayer_args': { 'displayer': 'obj' } }

class GalleryStyling(StylingAnalysis):
    requires_summary = ['dtype']
    pinned_rows = [obj_('dtype')]
class GalleryBuckaroo(PolarsBuckarooWidget):
    analysis_klasses = [GalleryStyling, PlTyping]

In [ ]:
obj_df = pl.DataFrame({
    'bools':[True, True, False, False, True, None],
    'ints': [   5,   20,    30,   -10, 7772, None],
    'timestamp':["2020-01-01 01:00Z", "2020-01-01 02:00Z", "2020-02-28 02:00Z", "2020-03-15 02:00Z", None, None],
    'dicts': [ {'a':10, 'b':20, 'c':'some string'}, None, None, None, None, None], #polars
    'nested_dicts': [{'level_1': {'a':10, 'b':20, 'c':'some string'}}, None, None, None, None, None],
    'lists': [['a','b'], [1,2], None, None, None, None],
    'lists-string': [['a','b'], ['foo', 'bar'], None, None, None, None],
    'lists-int': [[10, 20], [100, 500], [8], None, None, None]}
)
GalleryBuckaroo(obj_df)

In [ ]:
obj_df = pl.DataFrame({
    'bools':[True, True, False, False, True, None],
    'ints': [   5,   20,    30,   -10, 7772, None],
    'dicts': [ {'a':10, 'b':20, 'c':'some string'}, None, None, None, None, None], #polars
    'nested_dicts': [{'level_1': {'a':10, 'b':20, 'c':'some string'}}, None, None, None, None, None],
    #'nested_dicts2': pl.Series([{'level_1': {'a':10, 'b':20, 'c':'some string'}}, None, None, None, None, None], dtype=pl.Object)}
}
#    schema_overrides={'nested_dicts2':
)
GalleryBuckaroo(obj_df)

In [ ]:
bool_df = pl.DataFrame({'bools':[True, True, False, False, True, None]})
bool_df = bool_df.select(
    pl.col('bools').alias('bools_obj_displayer'), 
    pl.col('bools').alias('bools_boolean_displayer'))
#fix         #'bools_bool_checkbox_displayer':  {'displayer_args': {'displayer': 'boolean_checkbox'}
PolarsBuckarooWidget(
    bool_df, 
    column_config_overrides={
        'bools_obj_displayer':  {'displayer_args': {'displayer': 'obj'}},      
        'bools_boolean_displayer': {'displayer_args': {'displayer': 'boolean'}}})

In [ ]:
string_df = pl.DataFrame({'strings':["asdf", "qwerty", "really long string, much  much longer", None,  "A"]})
string_df = string_df.select(
    pl.col('strings').alias('strings_obj_displayer'), 
    pl.col('strings').alias('strings_string_displayer'),
    pl.col('strings').alias('strings_string_displayer_max_len'))
PolarsBuckarooWidget(
    string_df, 
    column_config_overrides={
        'strings_obj_displayer':  {'displayer_args': {'displayer': 'obj'}},      
        'strings_string_displayer': {'displayer_args': {'displayer': 'string'}},
        'strings_string_displayer_max_len': {'displayer_args': {'displayer': 'string', 'max_length':15}}
    })

In [ ]:
float_df = pl.DataFrame({
    'float': [5, -8, 13.23, -8.01, -999.345245234, None]})

def float_(min_digits, max_digits):
    return {'displayer_args': { 'displayer': 'float', 'min_fraction_digits':min_digits, 'max_fraction_digits': max_digits}}
float_df = float_df.select(
    pl.col('float').alias('float_obj_displayer'), 
    pl.col('float').alias('float_float_displayer_1__3'),
    pl.col('float').alias('float_float_displayer_0__3'),
    pl.col('float').alias('float_float_displayer_3__3'),
    pl.col('float').alias('float_float_displayer_3_13'))
PolarsBuckarooWidget(
    float_df,
    column_config_overrides={
        'float_obj_displayer':  {'displayer_args': {'displayer': 'obj'}},      
        'float_float_displayer_1__3' : float_(1,3),
        'float_float_displayer_0__3' : float_(0,3),
        'float_float_displayer_3__3' : float_(3,3),
        'float_float_displayer_3_13' : float_(3,13)})

In [ ]:
datetime_df = pl.DataFrame({'timestamp':["2020-01-01 01:00Z", "2020-01-01 02:00Z", "2020-02-28 02:00Z", "2020-03-15 02:00Z", None]})
datetime_df = datetime_df.select(
    pl.col('timestamp').alias('timestamp_obj_displayer'),
    pl.col('timestamp').alias('timestamp_datetime_default_displayer'),
    pl.col('timestamp').alias('timestamp_datetime_locale_en-US'),
    pl.col('timestamp').alias('timestamp_datetime_locale_en-US-Long'),
    pl.col('timestamp').alias('timestamp_datetime_locale_en-GB'))

def locale(locale, args={}):
    return {'displayer_args': {'displayer': 'datetimeLocaleString',
                                'locale':locale,
                                'args':args}}
PolarsBuckarooWidget(datetime_df,
    column_config_overrides={
        'timestamp_obj_displayer':  {'displayer_args': {'displayer': 'obj'}},    
        'timestamp_datetime_default_displayer' : {'displayer_args':  {  'displayer': 'datetimeDefault'}},
        'timestamp_datetime_locale_en-US' :locale('en-US'),
        'timestamp_datetime_locale_en-US-Long': locale('en-US', { 'weekday': 'long'}),
        'timestamp_datetime_locale_en-GB' : locale('en-GB')})

In [ ]:
link_df = DataFrame({'raw':      ['https://github.com/paddymul/buckaroo', 'https://github.com/pola-rs/polars'],
                    'linkify' : ['https://github.com/paddymul/buckaroo', 'https://github.com/pola-rs/polars']})
BUCKAROOWidget(link_df,
               column_config_overrides={'linkify': {'displayer_args':  {  'displayer': 'linkify'}}})
#fixme no underline or blue highlighting of links... but they are links

# Tooltips

In [ ]:
ROWS = 200
typed_df = pl.DataFrame({'int_col':np.random.randint(1,50, ROWS), 'float_col': np.random.randint(1,30, ROWS)/.7,
                         "str_col": ["foobar"]* ROWS})
PolarsBuckarooWidget(
    typed_df, 
    column_config_overrides={
        'str_col':
            {'tooltip_config': { 'tooltip_type':'simple', 'val_column': 'int_col'}}})

# Color_map_config
```ts
//ColorMapRules
export interface ColorMapRules {
  color_rule: 'color_map';
  map_name: 'BLUE_TO_YELLOW' | 'DIVERGING_RED_WHITE_BLUE';
  //optional, the column to base the ranges on.  the proper histogram_bins must still be sent in for that column
  val_column?: string;
}

//if exist_column is not null,  set cell style to condtional_color... used for highlighting changed values or errored_rows
export interface ColorWhenNotNullRules {
  color_rule: 'color_not_null';
  conditional_color: string | 'red';
  exist_column: string;
}

export interface ColorFromColumn {
  color_rule: 'color_from_column';
  col_name: string;
}
```

In [ ]:
PolarsBuckarooWidget(
    typed_df, 
    column_config_overrides={
        'float_col': {'color_map_config': {
          'color_rule': 'color_map',
          'map_name': 'BLUE_TO_YELLOW',
        }}})

In [ ]:
error_df = pl.DataFrame({
    'a': [10, 20, 30],
    'err_messages': [None, "a must be less than 19, it is 20", "a must be less than 19, it is 30"]})

PolarsBuckarooWidget(
    error_df, 
    column_config_overrides={
        'a': {'color_map_config': {
            'color_rule': 'color_not_null',
            'conditional_color': 'red',
            'exist_column': 'err_messages'}}})

In [ ]:
color_df = pl.DataFrame({
    'a': [10, 20, 30],
    'a_colors': ['red', '#d3a', 'green']})

PolarsBuckarooWidget(
    color_df, 
    column_config_overrides={
        'a': { 'color_map_config': {
          'color_rule': 'color_from_column',
          'col_name': 'a_colors'}}})

Extra col def type

`ag_grid_specs`
only to be used for very specific hacking.  You can set any primitive property, but can't set function props
https://github.com/ag-grid/ag-grid/blob/3cce76f1f2fd7142126f659fdc1dc3f0b7bc6191/grid-community-modules/core/src/ts/entities/colDef.ts#L4

latest
https://github.com/ag-grid/ag-grid/blob/latest/grid-community-modules/core/src/ts/entities/colDef.ts